In [1]:
import pandas as pd 
import numpy as np
from libtiff import TIFF 
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import Ridge,RidgeCV
from collections import Counter 
import os
import re 
import scipy.optimize
import sympy as sp 
from sklearn.metrics import mean_squared_error
import math
import statsmodels.api as sm 

In [2]:
#(the list to store the image name)：satellite_year 
file_path = 'E:\\tiff_picture\\DSPM_OLS'
file_pathh = r'E:\tiff_picture\DSPM_OLS'
NTL_image_file = os.listdir(file_pathh)
satelline_year = []
for i in NTL_image_file:
    name = re.findall('(.{7}).v4',i)
    satelline_year.append(name)
satelline_year = np.squeeze(satelline_year).tolist()
print(satelline_year)
print(NTL_image_file) 
F18 = ['F182011','F182012','F182013']
IMG = []
for i in range(len(NTL_image_file)):
    if str(satelline_year[i]) == 'F182010':
        locals()['IMG'+satelline_year[i]] = TIFF.open(os.path.join(file_path,satelline_year[i]+".v4d_web.stable_lights.avg_vis.tif"))
        IMG.append(i)
    elif satelline_year[i] in F18:
        locals()['IMG'+satelline_year[i]] = TIFF.open(os.path.join(file_path,satelline_year[i]+".v4c_web.stable_lights.avg_vis.tif"))
        IMG.append(i)
    elif satelline_year[i] =='F152000':
        locals()['IMG'+satelline_year[i]] = TIFF.open(os.path.join(file_path,satelline_year[i]+".v4b_web.stable_lights.avg_vis.tif"))
    else:
        locals()['IMG'+satelline_year[i]] = TIFF.open(os.path.join(file_path,satelline_year[i]+".v4b_web.stable_lights.avg_vis.tif"))
        IMG.append(i)

['F101992', 'F101993', 'F101994', 'F121994', 'F121995', 'F121996', 'F121997', 'F121998', 'F121999', 'F141998', 'F141999', 'F142000', 'F142001', 'F142002', 'F142003', 'F152000', 'F152001', 'F152002', 'F152003', 'F152004', 'F152005', 'F152006', 'F152007', 'F162004', 'F162005', 'F162006', 'F162007', 'F162008', 'F162009', 'F182010', 'F182011', 'F182012', 'F182013']
['F101992.v4b_web.stable_lights.avg_vis.tif', 'F101993.v4b_web.stable_lights.avg_vis.tif', 'F101994.v4b_web.stable_lights.avg_vis.tif', 'F121994.v4b_web.stable_lights.avg_vis.tif', 'F121995.v4b_web.stable_lights.avg_vis.tif', 'F121996.v4b_web.stable_lights.avg_vis.tif', 'F121997.v4b_web.stable_lights.avg_vis.tif', 'F121998.v4b_web.stable_lights.avg_vis.tif', 'F121999.v4b_web.stable_lights.avg_vis.tif', 'F141998.v4b_web.stable_lights.avg_vis.tif', 'F141999.v4b_web.stable_lights.avg_vis.tif', 'F142000.v4b_web.stable_lights.avg_vis.tif', 'F142001.v4b_web.stable_lights.avg_vis.tif', 'F142002.v4b_web.stable_lights.avg_vis.tif', 'F142

step 1：adjust the image from the same satellite(such as F18)
DNt-1 means the pixel value in (arr,col). And ,DNt means the pixel value in same point of latter image.
if DNt-1>DNt,DNt = DNt-1
if DNt-1<DNt,DNt=DNt
if DNt=0,DNt = 0

In [3]:
def get_new_tiff(x,y):
    #I only get the Asia part
    lat_s,lat_e,lon_s,lon_e = 2640,8520,30360,39600
    IMG1 = x[lat_s:lat_e,lon_s:lon_e]
    IMG2 = y[lat_s:lat_e,lon_s:lon_e]
    arr1 = np.array(IMG1)
    arr2 = np.array(IMG2)
    #IMG1 is equal to 0 or bigger than IMG2 get True
    #IMG1 is smaller than IMG2 get False
    mask = (arr1==0)  
    arr1_2 = arr1-arr2
    mask_2 = (arr1_2>=0) 
    adjust_1 = np.where(mask_2,arr1,arr2)  
    zeros_arr = np.zeros_like(arr1_2)
    result = np.where(mask,zeros_arr,adjust_1) 
    return result

In [4]:
# if the image is the last image of same satellite, it can't be adjusted
L1 = ['F12'+str(i) for i in range(1997,2000)]
L2 = ['F14'+str(i) for i in range(1999,2004)]
L3 = ['F15'+str(i) for i in range(2001,2008)]
L4 = ['F16'+str(i) for i in range(2005,2010)]
L5 = ['F18'+str(i) for i in range(2011,2014)]
L = L1+L2+L3+L4+L5 #which can be adjusted
all_piexl = []
for i in satelline_year:
    if (i in L) is True:
        name2 = str(i)[0:3]+str(int(i[3:7])-1)
        x = locals()['IMG'+i].read_image()
        y = locals()['IMG'+name2].read_image()
        adjust =get_new_tiff(x,y)
    else:
        pic = locals()['IMG'+i].read_image()
        adjust = pic[2640:8520,30360:39600]
    all_piexl.append(adjust)
# ##get the result into a list named all_piexl
all_piexl

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0

Considering that there are too few points and the function image is not smooth enough, we can train the model by constructing new points

In [5]:
#use y = kx+b to add some point：
def add_point(x1,x2,y1,y2):
    band_x = (x2-x1)/10
    band_y = (y2-y1)/10
    point=[]
    for i in range(1,10):
        new_x = x1+band_x*i
        new_y = y1+band_y*i 
        point.append([new_x,new_y])
    return point 

In [1]:
lon_lat = pd.read_excel(r'C:/一些文件/python_data/area_of_city/lon_lat.xlsx')
#lon_lat is my file of lat and lon of city center and city size
for p in range(0,33):
    tiff = all_piexl[p]
    name = satelline_year[p]
    lon_lat[name+'mean_new'] = np.nan 
    lon_lat[name+'mean'] = np.nan
    for i in range(len(lon_lat)):
        R = lon_lat['R_allcity'][i]/111 
        lat_city_s = (53-lon_lat['lat'][i]-R)/(30/3600)
        lat_city_e = (53-lon_lat['lat'][i]+R)/(30/3600)
        lon_city_s = (lon_lat['lon'][i]-R-73)/(30/3600)
        lon_city_e = (lon_lat['lon'][i]+R-73)/(30/3600)
        ma = tiff[int(lat_city_s):int(lat_city_e),int(lon_city_s):int(lon_city_e)]
        plt.imshow(ma)
        plt.xlabel(name+lon_lat['city'][i])
        plt.show()
        lon_lat[name+'mean'][i] = np.mean(ma)
        #for which don't have saruration part , which max is smaller than 63
#         if np.max(ma)<63:
#             ma_mean = np.mean(ma,axis=0)
#             ma_var = np.var_name(ma_mean)
#             lon_lat['var'][i] = ma_var
#             lon_lat[name[0]+'mean'][i] = np.mean(ma)
        if np.max(ma)>=63:
            df_DN = pd.DataFrame(ma).reset_index().rename(columns={'index':'arr'})
            df = df_DN.melt(id_vars='arr',value_name='\d',var_name='col').rename(columns={'\d':'DN'})
            PN = pd.DataFrame(pd.value_counts(df['DN'])).reset_index().rename(columns={'DN':'PN'})
            PN.sort_values(['index'],inplace=True)
            PN.rename(columns={'index':'DN'},inplace=True)
            PN.reset_index(inplace=True)
            PN.drop(columns=['index'],inplace=True)
            arr = np.matrix(PN['PN'][~PN['DN'].isin([0])]).T
            one = np.tril(np.ones(arr.shape[0]),0)
            arr_APN = np.dot(one,arr)
            APN = np.squeeze(arr_APN.tolist())
            DN_PN = PN[~PN['DN'].isin([0])]
            DN_PN.reset_index(inplace=True)
            DN_PN_APN = pd.DataFrame([DN_PN['DN'],DN_PN['PN'],APN]).T
            DN_PN_APN = DN_PN_APN.rename(columns={'Unnamed 0':'APN'})
            DN_PN_APN['APN'][0] = np.nan
            generate_point = []
            for n in range(2,len(DN_PN_APN)-1):
                x1 = DN_PN_APN['APN'][n]
                x2 = DN_PN_APN['APN'][n+1]
                y1 = DN_PN_APN['DN'][n]
                y2 = DN_PN_APN['DN'][n+1]
                L = add_point(x1,x2,y1,y2)
                generate_point.append(L)
            df_add_point = pd.DataFrame(generate_point[0]).rename(columns={0:'APN',1:'DN'})
            for k in range(1,len(generate_point)):
                df = pd.DataFrame(generate_point[k])
                df.rename(columns={0:'APN',1:'DN'},inplace=True)
                df_add_point = pd.concat([df_add_point,df])
            df_DN_APN_all = pd.concat([DN_PN_APN,df_add_point]) 
            df_DN_APN_all['APN_2'] = df_DN_APN_all['APN'].apply(lambda x : np.square(x))
            df_DN_APN_all['APN_3'] = df_DN_APN_all['APN'].apply(lambda x : np.power(x,3))
            df_DN_APN_all.sort_values('DN')
            df_DN_APN_all.reset_index(inplace=True)
            #drop duplicated ones
            L = [float('%.1f' % i) for i in np.arange(62,63,0.1)]
            reg_crow_data = df_DN_APN_all[~df_DN_APN_all['DN'].isin(L)]
            #I use OLS to get the cubic. And I also try liner-regression which also don't work.
            #and I want to get the curve fit the original curve as much as possible,so over fitting can be accepted 
            model = sm.formula.ols('DN~APN+APN_2+APN_3',data=reg_crow_data).fit()
            Int,a,b,c = model.params
            num_of_pixel = (DN_PN_APN['APN'][DN_PN_APN['DN'].isin([63])]).tolist()[0]
            max_DN = Int+a*num_of_pixel+b*num_of_pixel**2+c*num_of_pixel**3
            if int(max_DN) <= 63:
                lon_lat[name+'mean_new'][i] = 404
            else:
                list_APN = []
                for j in range(62,int(max_DN)+2):
                    x = sp.Symbol('x')
                    solution_of_equation = sp.solve([Int+a*x+b*x**2+c*x**3-j],[x])
                    L = [list(i) for i in solution_of_equation]
                    solution = [complex(i[0]).real for i in L]
                    if solution[2]<0:
                        solution.pop(2)
                    if solution[1]<0:
                        solution.pop(1)
                    if solution[0]<0:
                        solution.pop(0)
                    need_APN = np.mean(solution)
                    list_APN.append(need_APN)
                xx = [i for i in range(63,int(max_DN)+2)]
                PPN = pd.DataFrame([list_APN,xx]).T.rename(columns={0:'APN',1:'DN'})
                PN_last = pd.concat([DN_PN_APN[~DN_PN_APN['DN'].isin([63])],PPN])
                PN = PN_last['APN'].diff(periods=1)
                #PN is a Series
                PN_list = (PN.values).tolist()
        #         PN_list.pop(0)
        #         PN_list.append([np.nan]*2)
                PN_last['PN_2'] = PN_list
                last_last = PN_last[PN_last['PN_2']>0]
                last_last['PN'][last_last['PN'].isin([np.nan])] = last_last['PN_2'][last_last['PN'].isin([np.nan])]
                x = np.sum(last_last['PN'][last_last['PN']>0]*last_last['DN'][last_last['PN']>0])/np.sum(last_last['DN'][last_last['PN']>0])
                lon_lat[name+'mean_new'][i] = x 
        else:
            lon_lat[name+'mean_new'][i] = np.mean(ma)

In [8]:
# lon_lat.head()
# NTL = pd.DataFrame(lon_lat.drop(columns=['Unnamed: 0','lon','lat','市辖区','建成区','R_allcity','R_urban']))
# NTL.head()
lon_lat.to_excel(r'C:\一些文件\data\2020_11_09.xlsx')

In [ ]:
##then I use GDP and electry consumption to verify data quality
##The correlation is reduced。。。 
# So 